# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.

2003 has a lot less columns in common with the other years

2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [1]:
from project_data_Buske import common_columns

In [2]:
from project_data_Buske import latlong_to_code

In [3]:
#from project_data_Buske import latlong_to_name

In [4]:
from project_data_Buske import code_to_name

In [5]:
from project_data_Buske import code_to_distance

In [6]:
from project_data_Buske import latlong_to_distance

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [84]:
from glob import glob
files = glob('./data/MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]
[file for file in files]

['./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt']

4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [90]:
import pandas as pd
fileiterator = [pd.read_csv(file, chunksize = 500, sep = '|', dtype={'centroid_lat':str, 'centroid_long':str}) for file in files]
fileiterator

In [91]:
from toolz import first

In [77]:
firstchunk = first(fileiterator[0])
firstchunk.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739
1,0.0,0.93,NaN,NaN,N,NaN,24457.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2004,55005.0,NaN,45.41354,-93.2701
2,0.0,8.75,NaN,NaN,N,NaN,24442.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2004,55005.0,NaN,45.41318,-93.27344
3,0.0,11.17,NaN,NaN,N,NaN,410.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2004,55005.0,NaN,45.41167,-93.27684
4,0.0,14.46,NaN,NaN,N,NaN,480.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2004,55070.0,NaN,45.41169,-93.27849


In [78]:
from dfply import *
from more_dfply import *

In [79]:
exp = (firstchunk 
         >> select(common_columns)
         >> mutate(lat_long = pd.Series(zip(firstchunk.centroid_lat, firstchunk.centroid_long)))
         >> mutate(lake_code = recode(X.lat_long, latlong_to_code))
         >> mutate(lake_name = recode(X.lake_code, code_to_name))
         >> mutate(parcel_dist_to_lake = recode(X.lat_long, latlong_to_distance))
         #>> filter_by(X.parcel_dist_to_lake <= 1600)
      )         
exp

,OWN_ADD_L2,YEAR_BUILT,COOLING,GREEN_ACRE,PARC_CODE,EMV_TOTAL,SALE_DATE,TAX_ADD_L2,CITY_USPS,USE4_DESC,...,STREETTYPE,OWNER_NAME,centroid_lat,PREFIX_DIR,STREETNAME,OWN_ADD_L3,lat_long,lake_code,lake_name,parcel_dist_to_lake
0,BETHEL,1980.0,NaN,N,0.0,23964.0,NaN,BETHEL,NaN,NaN,...,NaN,NaN,45.41332,NaN,NaN,"MN, 55005","(45.41332, -93.26739)",02009100-03,NaN,5939.891324
1,BETHEL,1974.0,NaN,N,0.0,171837.0,NaN,BETHEL,BETHEL,NaN,...,ST,NaN,45.41354,NaN,DOGWOOD,"MN, 55005","(45.41354, -93.2701)",02009100-03,NaN,5939.891324
2,ST FRANCIS,1969.0,NaN,N,0.0,195751.0,2001-04-26,SIMI VALLEY,BETHEL,NaN,...,ST,NaN,45.41318,NaN,DOGWOOD,"MN, 55005","(45.41318, -93.27344)",02009100-03,NaN,5939.891324
3,BETHEL,1989.0,NaN,N,0.0,210338.0,1995-03-23,CHESTERFIELD,BETHEL,NaN,...,AVE,NaN,45.41167,NaN,245TH,"MN, 55005","(45.41167, -93.27684)",02009100-03,NaN,5939.891324
4,EAST BETHEL,1995.0,NaN,N,0.0,204359.0,1995-04-04,RICHMOND,BETHEL,NaN,...,AVE,NaN,45.41169,NaN,245TH,"MN, 55005","(45.41169, -93.27849)",02009100-03,NaN,5939.891324
5,ST PAUL,0.0,NaN,N,0.0,210000.0,NaN,ST PAUL,NaN,NaN,...,NaN,NaN,45.40981,NaN,NaN,"MN, 55155","(45.40981, -93.29885)",02009100-03,NaN,5939.891324
6,EAST BETHEL,0.0,NaN,N,0.0,10005.0,2004-06-25,COVINA,NaN,NaN,...,NaN,NaN,45.41172,NaN,NaN,"MN, 55005","(45.41172, -93.27973)",02009100-03,NaN,5939.891324
7,ISANTI,1969.0,NaN,N,0.0,169388.0,1987-02-01,COVINA,ISANTI,NaN,...,AVE,NaN,45.41168,NaN,245TH,"MN, 55040","(45.41168, -93.28033)",02009100-03,NaN,5939.891324
8,ST PAUL,0.0,NaN,N,0.0,21800.0,1995-06-07,ST PAUL,NaN,NaN,...,NaN,NaN,45.4117,NaN,NaN,"MN, 55101","(45.4117, -93.28091)",02009100-03,NaN,5939.891324
9,ST PAUL,0.0,NaN,N,0.0,247500.0,1994-12-14,ST PAUL,NaN,NaN,...,NaN,NaN,45.41162,NaN,NaN,"MN, 55155","(45.41162, -93.28364)",02009100-03,NaN,5939.891324


In [80]:
exp.lake_name.isna().sum()

382

In [81]:
exp.columns

Index(['OWN_ADD_L2', 'YEAR_BUILT', 'COOLING', 'GREEN_ACRE', 'PARC_CODE',
       'EMV_TOTAL', 'SALE_DATE', 'TAX_ADD_L2', 'CITY_USPS', 'USE4_DESC',
       'ZIP4', 'HEATING', 'Year', 'GARAGE', 'SCHOOL_DST', 'SALE_VALUE',
       'USE3_DESC', 'NUM_UNITS', 'PLAT_NAME', 'SPEC_ASSES', 'SUFFIX_DIR',
       'LANDMARK', 'BLDG_NUM', 'OWN_ADD_L1', 'WSHD_DIST', 'XUSE2_DESC', 'LOT',
       'EMV_LAND', 'COUNTY_ID', 'BASEMENT', 'DWELL_TYPE', 'USE1_DESC',
       'MULTI_USES', 'FIN_SQ_FT', 'Shape_Leng', 'TAX_NAME', 'ACRES_POLY',
       'HOMESTEAD', 'UNIT_INFO', 'AG_PRESERV', 'TOTAL_TAX', 'ACRES_DEED',
       'AGPRE_EXPD', 'BLOCK', 'GARAGESQFT', 'USE2_DESC', 'TAX_CAPAC',
       'PREFIXTYPE', 'XUSE1_DESC', 'AGPRE_ENRD', 'centroid_long', 'ZIP',
       'TAX_ADD_L3', 'CITY', 'TAX_EXEMPT', 'HOME_STYLE', 'TAX_ADD_L1',
       'XUSE3_DESC', 'XUSE4_DESC', 'OPEN_SPACE', 'EMV_BLDG', 'PIN',
       'Shape_Area', 'OWNER_MORE', 'STREETTYPE', 'OWNER_NAME', 'centroid_lat',
       'PREFIX_DIR', 'STREETNAME', 'OWN_ADD_L3', 

5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

In [92]:
def addcolumns(chunk):
    return (chunk 
             >> select(common_columns)
             >> mutate(lat_long = pd.Series(zip(chunk.centroid_lat, chunk.centroid_long)))
             >> mutate(lake_code = recode(X.lat_long, latlong_to_code))
             >> mutate(lake_name = recode(X.lake_code, code_to_name))
             >> mutate(parcel_dist_to_lake = recode(X.lat_long, latlong_to_distance))
             #>> filter_by(X.parcel_dist_to_lake <= 1600)
      )         

In [93]:
ch = first(fileiterator[2])
addcolumns(ch)

,OWN_ADD_L2,YEAR_BUILT,COOLING,GREEN_ACRE,PARC_CODE,EMV_TOTAL,SALE_DATE,TAX_ADD_L2,CITY_USPS,USE4_DESC,...,STREETTYPE,OWNER_NAME,centroid_lat,PREFIX_DIR,STREETNAME,OWN_ADD_L3,lat_long,lake_code,lake_name,parcel_dist_to_lake
0,ANDOVER,2000.0,NaN,NaN,0.0,357700.0,2000-11-17,ANDOVER,ANDOVER,NaN,...,ST,ODONNELL MICHAEL H & SHERRI L,45.22905,NaN,ALDER,"MN, 55304","(45.22905, -93.26607)",02005300-01,Ham Lake,4567.588683
1,ANDOVER,1999.0,NaN,NaN,0.0,358600.0,NaN,ANDOVER,ANDOVER,NaN,...,ST,MOSS WAYNE E & THERESA A,45.22892,NaN,ALDER,"MN, 55304","(45.22892, -93.26591)",02005300-01,Ham Lake,4567.588683
2,ANDOVER,1999.0,NaN,NaN,0.0,336300.0,NaN,ANDOVER,ANDOVER,NaN,...,ST,ARVIDSON LAWRENCE C & LINDA F,45.22864,NaN,ALDER,"MN, 55304","(45.22864, -93.26566)",02005300-01,Ham Lake,4567.588683
3,ANDOVER,1999.0,NaN,NaN,0.0,331300.0,NaN,ANDOVER,ANDOVER,NaN,...,ST,STRUSS HERBERT F & THIESE A,45.22829,NaN,ALDER,"MN, 55304","(45.22829, -93.26547)",02005300-01,Ham Lake,4567.588683
4,ANDOVER,2000.0,NaN,NaN,0.0,344600.0,NaN,ANDOVER,ANOKA,NaN,...,ST,WINDMEIER TRUSTEE/DAVID,45.22811,NaN,ALDER,"MN, 55304","(45.22811, -93.26548)",02005300-01,Ham Lake,4567.588683
5,ANDOVER,2002.0,NaN,NaN,0.0,338000.0,2002-07-26,ANDOVER,ANDOVER,NaN,...,ST,GREENE WILLIAM J & MARCIA R,45.22859,NaN,ALDER,"MN, 55304","(45.22859, -93.26641)",02005300-01,Ham Lake,4567.588683
6,ANDOVER,2002.0,NaN,NaN,0.0,335500.0,2002-07-29,ANDOVER,ANDOVER,NaN,...,ST,STELLMACH CONSTANCE A,45.22827,NaN,ALDER,"MN, 55304","(45.22827, -93.26625)",02005300-01,Ham Lake,4567.588683
7,ANDOVER,2002.0,NaN,NaN,0.0,286500.0,2003-03-13,ANDOVER,ANDOVER,NaN,...,AVE,QUIGLEY MICHAEL F & LINDA A,45.2287,NaN,142ND,"MN, 55304","(45.2287, -93.26747)",02005300-01,Ham Lake,4567.588683
8,ANDOVER,2001.0,NaN,NaN,0.0,237500.0,2002-04-09,ANDOVER,ANDOVER,NaN,...,AVE,WILLIAMS WILLIAM R & CAROL A,45.22878,NaN,142ND,"MN, 55304","(45.22878, -93.26704)",02005300-01,Ham Lake,4567.588683
9,ANDOVER,2002.0,NaN,NaN,0.0,254100.0,2002-11-22,ANDOVER,ANDOVER,NaN,...,LN,SANDERS JAMES E & JUDY E,45.22806,NaN,141ST,"MN, 55304","(45.22806, -93.26731)",02005300-01,Ham Lake,4567.588683


In [21]:
addcolumns(firstchunk)

,OWN_ADD_L2,YEAR_BUILT,COOLING,GREEN_ACRE,PARC_CODE,EMV_TOTAL,SALE_DATE,TAX_ADD_L2,CITY_USPS,USE4_DESC,...,STREETTYPE,OWNER_NAME,centroid_lat,PREFIX_DIR,STREETNAME,OWN_ADD_L3,lat_long,lake_code,lake_name,parcel_dist_to_lake
0,ELK RIVER,1996.0,NaN,NaN,0.0,0.0,1899-12-30,ELK RIVER,SAINT FRANCIS,NaN,...,ST,DESCHENES TRUSTEE JAMES & SNEEN TRUSTEE ARTHUR,45.40663,NaN,QUICKSILVER,"MN, 55330","(45.40663, -93.42361)",02013000-01,Pickerel Lake,4345.023767
1,ST FRANCIS,1998.0,NaN,NaN,0.0,563800.0,1899-12-30,ST FRANCIS,SAINT FRANCIS,NaN,...,ST,HEIFORT ALAN W & JULIE A,45.40537,NaN,QUICKSILVER,"MN, 55070","(45.40537, -93.42046)",02013000-01,Pickerel Lake,4345.023767
2,ST FRANCIS,1987.0,NaN,NaN,0.0,299800.0,1899-12-30,ST FRANCIS,SAINT FRANCIS,NaN,...,BLVD,JOHNSON DALE R & VICKIE E,45.40447,NaN,AMBASSADOR,"MN, 55070","(45.40447, -93.41845)",02013000-01,Pickerel Lake,4345.023767
3,ST FRANCIS,1989.0,NaN,NaN,0.0,243000.0,1899-12-30,ST FRANCIS,SAINT FRANCIS,NaN,...,BLVD,HELD ROBERT J & BARBARA I,45.40339,NaN,AMBASSADOR,"MN, 55070","(45.40339, -93.41675)",02013000-01,Pickerel Lake,4345.023767
4,ST FRANCIS,1994.0,NaN,NaN,0.0,286300.0,1899-12-30,ST FRANCIS,SAINT FRANCIS,NaN,...,BLVD,RATHBUN VIC F TRACY J,45.40122,NaN,AMBASSADOR,"MN, 55070","(45.40122, -93.41084)",02009100-02,NaN,2507.626106
5,ST FRANCIS,1992.0,NaN,NaN,0.0,302900.0,1899-12-30,ST FRANCIS,SAINT FRANCIS,NaN,...,DR,HARRIS MICHAEL P & COLLEEN A,45.40235,NaN,SEELYE BROOK,"MN, 55070","(45.40235, -93.41291)",02009100-02,NaN,2507.626106
6,ELK RIVER,0.0,NaN,NaN,0.0,5600.0,1899-12-30,ELK RIVER,SAINT FRANCIS,NaN,...,NaN,ZIMMERMAN CASEY D & G A,45.4028,NaN,NaN,"MN, 55330","(45.4028, -93.41497)",02009100-02,NaN,2507.626106
7,ST. FRANCIS,1988.0,NaN,NaN,0.0,293000.0,2007-01-19,ST. FRANCIS,SAINT FRANCIS,NaN,...,BLVD,MAY DONALD,45.40234,NaN,AMBASSADOR,"MN, 55070","(45.40234, -93.4167)",02013000-01,Pickerel Lake,4345.023767
8,ELK RIVER,1988.0,NaN,NaN,0.0,296300.0,1899-12-30,ELK RIVER,ELK RIVER,NaN,...,ST,WATROBA MICHAEL & SANDRA,45.39726,NaN,NACRE,"MN, 55330","(45.39726, -93.45262)",02013000-01,Pickerel Lake,4345.023767
9,ELK RIVER,0.0,NaN,NaN,0.0,106400.0,1899-12-30,ELK RIVER,ELK RIVER,NaN,...,NaN,WATROBA MICHAEL & SANDRA,45.39637,NaN,NaN,"MN, 55330","(45.39637, -93.45264)",02013000-01,Pickerel Lake,4345.023767


6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PID`.  Explain why this should make a proper primary key for the combined data. <br>

In [ ]:
for i, chunk in enumerate(fileiterator):
    for 

In [48]:
[(col, exp[col].is_unique) for col in exp]

[('OWN_ADD_L2', False),
 ('YEAR_BUILT', False),
 ('COOLING', False),
 ('GREEN_ACRE', False),
 ('PARC_CODE', False),
 ('EMV_TOTAL', False),
 ('SALE_DATE', False),
 ('TAX_ADD_L2', False),
 ('CITY_USPS', False),
 ('USE4_DESC', False),
 ('ZIP4', False),
 ('HEATING', False),
 ('Year', False),
 ('GARAGE', False),
 ('SCHOOL_DST', False),
 ('SALE_VALUE', False),
 ('USE3_DESC', False),
 ('NUM_UNITS', False),
 ('PLAT_NAME', False),
 ('SPEC_ASSES', False),
 ('SUFFIX_DIR', False),
 ('LANDMARK', False),
 ('BLDG_NUM', False),
 ('OWN_ADD_L1', False),
 ('WSHD_DIST', False),
 ('XUSE2_DESC', False),
 ('LOT', False),
 ('EMV_LAND', False),
 ('COUNTY_ID', False),
 ('BASEMENT', False),
 ('DWELL_TYPE', False),
 ('USE1_DESC', False),
 ('MULTI_USES', False),
 ('FIN_SQ_FT', False),
 ('Shape_Leng', False),
 ('TAX_NAME', False),
 ('ACRES_POLY', False),
 ('HOMESTEAD', False),
 ('UNIT_INFO', False),
 ('AG_PRESERV', False),
 ('TOTAL_TAX', False),
 ('ACRES_DEED', False),
 ('AGPRE_EXPD', False),
 ('BLOCK', False),
 ('

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PID`) to a given chunk.

#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.